# YOLO Step 3 v.3 - withOUT Synthetic data

Roboflow data generated:
2022-03-09 No augmentation
Version 4
Generated Mar 9, 2022

In [10]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import io
import pandas as pd
import json
import os
import glob 


In [11]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pvpFbwxA6FFS6pAwYdNk")
project = rf.workspace("new-workspace-s32a1").project("autoarki")
dataset = project.version(4).download("yolov5")

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 67 kB 6.8 MB/s eta 0:00:011
     |████████████████████████████████| 138 kB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 112.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 112.4 MB/s eta 0:00:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=6b93f1491b52d3b66781e70d36fde8e1fb31524202535d03f9265eaad1354b5c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
  Attempting uninstall: kiwisolver
    Found existing installation: kiwisolver 1.3.2
    Uninstalling kiwisolver-1.3.2:
      Successfully uninstalled kiwisolver-1.3.2
  Attempting uninstall: cycler
    Found existing installation: cycler 0.11.0
    Uninstalling cycler-0.11.0:
      Successfully uninstalled cycler-0.11.0
ERROR: pip's dependency resolver does n

Extracting Dataset Version Zip to autoarki-4 in yolov5pytorch:: 100%|██████████| 2080/2080 [00:00<00:00, 12430.90it/s]


In [23]:
# # change directory and copy files locally -- only do once
# # Roboflow output
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

!cp train/images/*  train --recursive
!cp train/labels/*  train --recursive
!rm -R train/labels; rm -R train/images

!cp  valid/images/*  valid --recursive
!cp  valid/labels/*  valid --recursive
!rm -R valid/labels; rm -R valid/images

!cp  test/images/*  test --recursive
!cp  test/labels/*  test --recursive
!rm -R test/labels; rm -R test/images

# Check 

In [24]:
def count_classes(file_path):
    '''
    increment class_counts dictionary for each instance of a 
    class check against roboflow summary (health check)
    
    '''
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_counts[og_line[0]] +=1
    return class_counts

In [25]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0}

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 3, '1': 126, '2': 18, '3': 1586, '4': 193, '5': 199, '6': 0}


# Replace Class Type

In [26]:
classes = {'1': '0', #Column_Wall_Intersection
           '3': '1', #illegibletext'
           '4': '2', #'no_stair_direction'
           '5': '3'  #missing_wall
          } 


In [27]:
def replace_class_n_with_class_name(file_path):
    '''for a txt file replace the class number with a class name'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n == '0' or class_n == '2': #bad annotation needs to be removed and CMU wall hinge insufficient data
                continue
            class_str = classes[class_n]
            new_line = og_line.replace(class_n, class_str, 1)
            new_lines.append(new_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [29]:
# replace class names for test and validation txt files

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)
    
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/valid")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)    
    

# Learning

use smallest YOLO model to start yolov5s.pt

In [30]:
import torch
import torchvision

In [31]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/")
#!pip install -r requirements.txt # first run
!python train.py\
    --data custom_dataset.yaml \
    --epochs 75\
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 5 \
    --weights yolov5s.pt 

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=75, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=5, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 11 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.2+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, l

No stair direction Class 2 is yielding low results. I'm guessing this has to do with tight bounds on this images. Because arrows or chinese characters nearby can indicate direction. 

# Remove Class 2

In [35]:
def remove_class_2(file_path):
    '''remove class 2'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n == '2': #Stairs
                continue
            new_lines.append(og_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [36]:
# replace class names for test and validation txt files

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    remove_class_2(file)
    
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/valid")
for file in glob.glob("*.txt"):
    remove_class_2(file)    
    

In [37]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0}

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 126, '1': 1586, '2': 0, '3': 199, '4': 0, '5': 0, '6': 0}


In [38]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/")
!python train.py\
    --data custom_dataset.yaml \
    --epochs 75\
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 5 \
    --weights yolov5s.pt 

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=custom_dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=75, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=5, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 11 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.10.2+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, l

In [39]:
!pip install wandb
!wandb login

Defaulting to user installation because normal site-packages is not writeable
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [40]:
import wandb

wandb.init(project="my-test-project", entity="kbaumstan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/")
!python train.py\
    --data custom_dataset.yaml \
    --epochs 75\
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 5 \
    --weights yolov5s.pt 

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        206        548      0.403      0.481      0.279      0.131

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     18/74     4.32G   0.05179   0.02097  0.005079        69       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        206        548      0.508      0.242      0.115     0.0539

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     19/74     4.32G   0.05038   0.02278  0.004991        85       640:  82%|███